# Module 2 Project Delphi Football Data

For this project, our goal is to 
- get the data we think is important from this SQL database,
- do some calculations and data transformation, 
- and then store everything in a MongoDB database.

Let's get into the specifics of this project.

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this kaggle page. Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database.
Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:
* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season.


At a Minimum:

- Query the SQL database
- Calculate summary statistics
- Get the weather data from the DarkSky API
- Load the data into MongoDB

## Importing the libraries

In [175]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

import dotenv
from dotenv import load_dotenv
import requests



## Making a Query to the SQL Database

Make a Query variable that I can select what I want from the relevant table in the database. 
Then Using pd.read_sql_query, pandify that table.

In [176]:
query = ('''SELECT FTHG, HomeTeam as TeamName
            FROM Matches
            WHERE Season = '2011' and Div != 'E0' ''')
df1 = pd.read_sql_query(query, conn)
df1

query2 = ('''SELECT FTAG, AwayTeam as TeamName
            FROM Matches
            WHERE Season = '2011' and Div != 'E0' ''')
df2 = pd.read_sql_query(query2, conn)
df2

Tot_goals = df1.join(df2.set_index('TeamName'), on='TeamName')
Tot_goals['total_goals'] = Tot_goals['FTHG'] + Tot_goals['FTAG']
Tot_goals

dateQuery = ('''SELECT Date, FTHG, HomeTeam as TeamName
            FROM Matches
            WHERE Season = '2011' and Div != 'E0' ''')
df3 = pd.read_sql_query(dateQuery, conn)

# Question 1:
## The name of the team AND The total number of goals scored by the team during the 2011 season

### Using SQL to query for the necessary data relevent to this investigation: 

In [177]:
# query = ('''SELECT ut.TeamName, sum(m.FTHG + m.FTAG) AS Tot_goals
#             FROM Matches m
#             JOIN Unique_Teams ut 
#             ''')

df = pd.read_sql_query(query, conn)
df.head()

,FTHG,TeamName
0,0,Nurnberg
1,1,Stuttgart
2,0,Wolfsburg
3,3,Mainz
4,0,Freiburg


Lets check the null values in this Subset

In [178]:
df.isna().sum()

FTHG        0
TeamName    0
dtype: int64

In [179]:
 df.duplicated().sum()

434

Perfect lets create some aggregate variables using this data:


In [180]:
df

,FTHG,TeamName
0,0,Nurnberg
1,1,Stuttgart
2,0,Wolfsburg
3,3,Mainz
4,0,Freiburg
...,...,...
607,1,Paderborn
608,1,Bochum
609,1,Ein Frankfurt
610,0,Aachen


In [34]:
d

array(['Nurnberg', 'Stuttgart', 'Wolfsburg', 'Mainz', 'Freiburg',
       "M'gladbach", 'Hamburg', 'Werder Bremen', 'Schalke 04', 'Hannover',
       'Hoffenheim', 'Leverkusen', 'Kaiserslautern', 'Hertha', 'Augsburg',
       'FC Koln', 'Dortmund', 'Bayern Munich', 'Hansa Rostock', 'Bochum',
       'Ein Frankfurt', 'Paderborn', 'Greuther Furth',
       'Fortuna Dusseldorf', 'Ingolstadt', 'Cottbus', 'St Pauli',
       'Frankfurt FSV', 'Union Berlin', 'Munich 1860', 'Aachen',
       'Karlsruhe', 'Braunschweig', 'Erzgebirge Aue', 'Duisburg',
       'Dresden'], dtype=object)

# Testing to_dict on the dataframe results

In [48]:
ddf = df.to_dict()
ddf.k

In [51]:
ddf.keys()

dict_keys(['Match_ID', 'Div', 'Season', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'])

# Api getter


In [ ]:
API : URL https://api.darksky.net/forecast/1943c3bf2e7ecf55aa2640f7b86097b8/37.8267,-122.4233

In [96]:
Key = '1943c3bf2e7ecf55aa2640f7b86097b8'

In [136]:
#Berlin
longitude = 13.4050
latitude = 52.5200

In [147]:
df3['Date'][0]
from datetime import datetime

In [173]:
# Needs alot of work

dates = pd.to_datetime(df3['Date'])
date = dates[0]

date = int(datetime.timestamp(date))
date

1333148400

In [174]:
#Time machine request from Dark sky
#Returns 8-12 Responses.... Why?

r = requests.get(f'https://api.darksky.net/forecast/{Key}/{latitude},{longitude},{date}').json()

len(r)

8

# Khairul SQL Code for Total Goals

In [168]:
query4 = """SELECT Team_name, SUM(Total_goals) Total_goals
           FROM 
                (
                 SELECT ut.TeamName Team_name,
                        SUM(m.FTHG) Total_goals
                 FROM unique_teams ut 
                 JOIN matches m
                 ON ut.TeamName = m.HomeTeam
                 WHERE m.Season == 2011 
                 GROUP BY Team_name 
                 UNION ALL
                 SELECT ut.TeamName Team_name,
                        SUM(m.FTAG) Total_goals
                 FROM unique_teams ut 
                 JOIN matches m
                 ON ut.TeamName = m.AwayTeam
                 WHERE m.Season == 2011
                 GROUP BY Team_name 
                )
           GROUP BY Team_name 
           """    

In [171]:
dfcheck = pd.read_sql_query(query4, conn)
dfcheck

,Team_name,Total_goals
0,Aachen,30
1,Arsenal,74
2,Aston Villa,37
3,Augsburg,36
4,Bayern Munich,77
5,Blackburn,48
6,Bochum,41
7,Bolton,46
8,Braunschweig,37
9,Chelsea,65


# Loading to MongoDB


In [185]:
python -m pip install pymongo

SyntaxError: invalid syntax (<ipython-input-185-e2a04d47df7b>, line 1)

In [184]:
import pymongo
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")

# grab a database from your server 
mydb = myclient['example_data']

ModuleNotFoundError: No module named 'pymongo'